# Coursera Capstone:Shwarma Restaurant Location Recommender

This project uses Foursquare API to recommend the neighborhood with the least frequency of shwarma restaurants in order to open a restaurant there.

### Importing Libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
#from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


Uploading an excel file with Abu Dhabi Neighborhoods along with their coordinates

In [5]:
from google.colab import files 
import io
uploaded = files.upload() #Choose the neighborhood file to be uploaded

Saving ADNeighborhoods.xlsx to ADNeighborhoods.xlsx


putting neighborhoods in a dataframe

In [6]:
ADN_frame=pd.read_excel('ADNeighborhoods.xlsx')
ADN_frame.head()

,Neighborhood,Lat,Lng
0,Al Aman,24.431478,54.432774
1,Al Bateen East,24.459207,54.336041
2,Al Bateen West,24.448197,54.355103
3,Al Falah,24.443217,54.727965
4,Al Lulu Island,24.500366,54.344003


counting neighborhoods

In [7]:

print('The dataframe has {} neighborhoods.'.format(
        len(ADN_frame['Neighborhood'].unique())
    )
)

The dataframe has 38 neighborhoods.


define Abu Dhabi coordinates

In [8]:
address = 'Abu Dhabi, AD'
latitude = 24.453900
longitude = 54.377300
print('The geograpical coordinate of Abu Dhabi are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Abu Dhabi are 24.4539, 54.3773.


showing neighborhoods on a folium map

In [12]:
# create map of New York using latitude and longitude values
AD_map = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(ADN_frame['Lat'], ADN_frame['Lng'], ADN_frame['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(AD_map)  
    
AD_map

define Foursquare parameters

In [0]:
CLIENT_ID = 'ZHRK11M3F34S5JBLH3K3MYS55VVTL5IAPJVEVWF4AQR54RGB' # your Foursquare ID
CLIENT_SECRET = 'BD1NT4KQVQETO4EWWAHUJS04VTARRUSW4ZXB15YZN0EJ40UY' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 30
radius=500

define a function to extract venues for each neighborhood

In [0]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

calling the venues function

In [16]:
AD_Venues = getNearbyVenues(names=ADN_frame['Neighborhood'],
                                   latitudes=ADN_frame['Lat'],
                                   longitudes=ADN_frame['Lng']
                                  )

Al Aman
Al Bateen East
Al Bateen West
Al Falah
Al Lulu Island
Al Maryah Island
Al Manaseer
Al Manhal
Al Maqtaa
Al Markaziyah
Al Meena
Al Moroor
Al Mushrif
Al Muzoon
Al Nahyan
Al Ras Al Akhdar
Al Reef
Al Reem Island
Al Rowdah
Al Shamkha
Bani Yas
Al Zaab
Al Zahiyah
Al Khalidyah
Bain Al Jisrain
Hideriyyat
Khalifa City
Marina Village
Masdar City
Mohammed Bin Zayed City
Saadiyat Island
Shakhbout City
Officers City
Qasr El Bahr
Qasr El Shatie
Yas Island
Al Samha
Zayed City


showing venues

In [17]:
print(AD_Venues.shape)
AD_Venues.head()

(373, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Al Aman,24.431478,54.432774,Zattara (زتارا),24.432171,54.434108,Bakery
1,Al Aman,24.431478,54.432774,Shakespeare and Co. شكسبير أند كو,24.431984,54.435373,Café
2,Al Aman,24.431478,54.432774,Mosaic (موزاييك),24.431384,54.436168,Lebanese Restaurant
3,Al Aman,24.431478,54.432774,Starbucks (ستاربكس),24.431905,54.435571,Coffee Shop
4,Al Aman,24.431478,54.432774,Muscle Fuel Healthy Eating (مصل فيول الأكل الصحي),24.431876,54.434552,Fast Food Restaurant


group results by neighborhood and show the total number of venues per neighborhood

In [18]:

AD_Venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Al Aman,30,30,30,30,30,30
Al Bateen East,4,4,4,4,4,4
Al Bateen West,4,4,4,4,4,4
Al Falah,1,1,1,1,1,1
Al Khalidyah,28,28,28,28,28,28
Al Manaseer,4,4,4,4,4,4
Al Manhal,6,6,6,6,6,6
Al Maqtaa,12,12,12,12,12,12
Al Markaziyah,18,18,18,18,18,18


showing the number of venue categories

In [19]:
print('There are {} uniques categories.'.format(len(AD_Venues['Venue Category'].unique())))

There are 111 uniques categories.


Analyze each neighborhood

In [21]:
# one hot encoding
AD_onehot = pd.get_dummies(AD_Venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
AD_onehot['Neighborhood'] = AD_Venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [AD_onehot.columns[-1]] + list(AD_onehot.columns[:-1])
AD_onehot = AD_onehot[fixed_columns]

AD_onehot.head()

,Neighborhood,Afghan Restaurant,Airport Terminal,American Restaurant,Asian Restaurant,Athletics & Sports,Australian Restaurant,Bakery,Bar,Beach,Bed & Breakfast,Bistro,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Burger Joint,Bus Station,Business Service,Cafeteria,Café,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Convenience Store,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Diner,Donut Shop,Electronics Store,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Flower Shop,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,General Entertainment,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Hookah Bar,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kids Store,Korean Restaurant,Lebanese Restaurant,Lighthouse,Liquor Store,Lounge,Market,Massage Studio,Medical Center,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Moroccan Restaurant,Movie Theater,Multiplex,Nail Salon,Nightclub,Park,Pedestrian Plaza,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Pool,Residential Building (Apartment / Condo),Resort,Restaurant,Sandwich Place,Seafood Restaurant,Shawarma Place,Shoe Store,Shopping Mall,Smoke Shop,Soccer Field,Soccer Stadium,Spa,Stables,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theme Restaurant,Toy / Game Store,Trail,Turkish Restaurant,Vietnamese Restaurant,Women's Store,Yoga Studio
0,Al Aman,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Al Aman,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Al Aman,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Al Aman,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Al Aman,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


grouping rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [84]:
AD_grouped = AD_onehot.groupby('Neighborhood').mean().reset_index()
AD_grouped.head()

,Neighborhood,Afghan Restaurant,Airport Terminal,American Restaurant,Asian Restaurant,Athletics & Sports,Australian Restaurant,Bakery,Bar,Beach,Bed & Breakfast,Bistro,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Burger Joint,Bus Station,Business Service,Cafeteria,Café,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Convenience Store,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Diner,Donut Shop,Electronics Store,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Flower Shop,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,General Entertainment,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Hookah Bar,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kids Store,Korean Restaurant,Lebanese Restaurant,Lighthouse,Liquor Store,Lounge,Market,Massage Studio,Medical Center,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Moroccan Restaurant,Movie Theater,Multiplex,Nail Salon,Nightclub,Park,Pedestrian Plaza,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Pool,Residential Building (Apartment / Condo),Resort,Restaurant,Sandwich Place,Seafood Restaurant,Shawarma Place,Shoe Store,Shopping Mall,Smoke Shop,Soccer Field,Soccer Stadium,Spa,Stables,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theme Restaurant,Toy / Game Store,Trail,Turkish Restaurant,Vietnamese Restaurant,Women's Store,Yoga Studio
0,Al Aman,0.0,0.0,0.000000,0.0,0.0,0.0,0.066667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.033333,0.066667,0.0,0.00,0.00,0.200000,0.033333,0.000000,0.0,0.0,0.066667,0.0,0.00,0.0,0.0,0.0,0.1,0.0,0.000000,0.0,0.0,0.133333,0.000000,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.000000,0.033333,0.00,0.0,0.033333,0.033333,0.0,0.0,0.0,0.00,0.000000,0.000000,0.0,0.0,0.0,0.0,0.066667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.00,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.033333,0.00,0.033333,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.033333,0.0,0.0,0.033333,0.0,0.0,0.000000,0.0,0.0,0.0
1,Al Bateen East,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.25,0.00,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.0,0.0,0.0,0.00,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.25,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.25,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0
2,Al Bateen West,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.00,0.25,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.000000,0.000000,0.25,0.0,0.000000,0.000000,0.0,0.0,0.0,0.25,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.00,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0
3,Al Falah,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.00,0.00,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.00,0.0,1.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.0,0.0,0.0,0.00,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.00,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0000

return top 5 venues by frequency for each neighborhood

In [83]:
num_top_venues = 10

for hood in AD_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = AD_grouped[AD_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Al Aman----
                  venue  freq
0                  Café  0.20
1  Fast Food Restaurant  0.13
2          Dessert Shop  0.10
3                Bakery  0.07
4   Lebanese Restaurant  0.07
5           Coffee Shop  0.07
6          Burger Joint  0.07
7                   Gym  0.03
8      Theme Restaurant  0.03
9    Chinese Restaurant  0.03


----Al Bateen East----
                 venue  freq
0       Cosmetics Shop  0.25
1       Sandwich Place  0.25
2            Nightclub  0.25
3     Business Service  0.25
4    Afghan Restaurant  0.00
5  Moroccan Restaurant  0.00
6          Pizza Place  0.00
7             Pharmacy  0.00
8            Pet Store  0.00
9     Pedestrian Plaza  0.00


----Al Bateen West----
                    venue  freq
0  Furniture / Home Store  0.25
1            Intersection  0.25
2    Gym / Fitness Center  0.25
3               Cafeteria  0.25
4       Afghan Restaurant  0.00
5             Pizza Place  0.00
6                Pharmacy  0.00
7               Pet Store  0.

creating a dataframe

In [0]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

creating a dataframe with the top 10 venues

In [88]:
 num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{}_common_venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th_common_venue'.format(ind+1))

# create a new dataframe
sorted_venues = pd.DataFrame(columns=columns)
sorted_venues['Neighborhood'] = AD_grouped['Neighborhood']

for ind in np.arange(AD_grouped.shape[0]):
    sorted_venues.iloc[ind, 1:] = return_most_common_venues(AD_grouped.iloc[ind, :], num_top_venues)

sorted_venues.head()

,Neighborhood,1st_common_venue,2nd_common_venue,3rd_common_venue,4th_common_venue,5th_common_venue,6th_common_venue,7th_common_venue,8th_common_venue,9th_common_venue,10th_common_venue
0,Al Aman,Café,Fast Food Restaurant,Dessert Shop,Bakery,Burger Joint,Lebanese Restaurant,Coffee Shop,Gym,Tea Room,Hotel
1,Al Bateen East,Cosmetics Shop,Nightclub,Sandwich Place,Business Service,French Restaurant,Department Store,Dessert Shop,Diner,Donut Shop,Electronics Store
2,Al Bateen West,Intersection,Gym / Fitness Center,Cafeteria,Furniture / Home Store,Fried Chicken Joint,Department Store,Dessert Shop,Diner,Donut Shop,Electronics Store
3,Al Falah,Deli / Bodega,Frozen Yogurt Shop,Department Store,Dessert Shop,Diner,Donut Shop,Electronics Store,Farmers Market,Fast Food Restaurant,Filipino Restaurant
4,Al Khalidyah,Café,Fast Food Restaurant,Coffee Shop,Pizza Place,Italian Restaurant,Spa,Park,Hookah Bar,Mexican Restaurant,Chocolate Shop


return the neighborhood where shwarma restaurant is not in the top 10 venues

In [89]:
sorted_venues[(sorted_venues["1st_common_venue"].str.contains("Restaurant")==False)	& (sorted_venues["2nd_common_venue"].str.contains("Restaurant")==False)	& (sorted_venues["3rd_common_venue"].str.contains("Restaurant")==False)	& (sorted_venues["4th_common_venue"].str.contains("Restaurant")==False)	& (sorted_venues["5th_common_venue"].str.contains("Restaurant")==False)	& (sorted_venues["6th_common_venue"].str.contains("Restaurant")==False)	& (sorted_venues["7th_common_venue"].str.contains("Restaurant")==False)	& (sorted_venues["8th_common_venue"].str.contains("Restaurant")==False)	& (sorted_venues["9th_common_venue"].str.contains("Restaurant")==False)	& (sorted_venues["10th_common_venue"].str.contains("Restaurant")==False)]

,Neighborhood,1st_common_venue,2nd_common_venue,3rd_common_venue,4th_common_venue,5th_common_venue,6th_common_venue,7th_common_venue,8th_common_venue,9th_common_venue,10th_common_venue
2,Al Bateen West,Intersection,Gym / Fitness Center,Cafeteria,Furniture / Home Store,Fried Chicken Joint,Department Store,Dessert Shop,Diner,Donut Shop,Electronics Store
5,Al Manaseer,Playground,Fried Chicken Joint,Café,Cafeteria,Department Store,Dessert Shop,Diner,Donut Shop,Electronics Store,Farmers Market
6,Al Manhal,Café,Jewelry Store,Hotel,Coffee Shop,Medical Center,Yoga Studio,Fried Chicken Joint,Dessert Shop,Diner,Donut Shop
15,Al Reef,Pool,Coffee Shop,Gym,Pizza Place,Tennis Court,Convenience Store,Residential Building (Apartment / Condo),Pet Store,Gymnastics Gym,Grocery Store
29,Qasr El Shatie,Beach,Pedestrian Plaza,Café,Yoga Studio,Frozen Yogurt Shop,Dessert Shop,Diner,Donut Shop,Electronics Store,Farmers Market


## Conclusion

In Abu Dhabi, there are 5 neighborhoods where restaurants are not in the top 10 venues, and those neighborhoods would be recommended for a restaurant business.